In [117]:
from requests_html import HTMLSession
import regex as re
import string
from mylog import MyLog as mylog
import os

In [113]:
logger = mylog()
htmlsession = HTMLSession()
headers = {
    'user-agent':
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:54.0) Gecko/20100101 Firefox/54.0',
    'accept':
    'text/html, application/xhtml+xml, application/xml; q=0.9, image/webp, */*; q=0.8',
    'referer':
    'https://www.1kkk.com'
}

In [18]:
# logger.info("test")
# logger.error("error test")

In [24]:
start_url = "https://www.1kkk.com"

In [20]:
comic_id = 17241

In [25]:
category_url = "".join([start_url, "/manhua", str(comic_id)])
category_url

'https://www.1kkk.com/manhua17241'

In [75]:
res = htmlsession.get(category_url)

## 获取章节urls

In [30]:
pattern_chapter = re.compile('https:\/\/www\.1kkk\.com\/ch(\d+)-\d+\/')

In [32]:
urls_chapter = []

In [41]:
for url in res.html.absolute_links:
    result_re = pattern_chapter.match(url)
    if result_re:
        urls_chapter.append((url, result_re.group(1)))
urls_chapter = list(set(urls_chapter))
urls_chapter.sort(key=lambda x: int(x[1]))

In [45]:
urls_chapter # urls_chapter中混有其他漫画的部分章节, 需要根据标题进行筛选
urls_chapter[0]

('https://www.1kkk.com/ch1-429512/', '1')

## 获取标题

In [76]:
tag_title = res.html.find('p.title')[1].text
comic_title, comic_score = tag_title.split()
print(comic_title, comic_score)

寄宿学校的朱丽叶 7.3分


## download单个章节

In [81]:
target_url = urls_chapter[1][0]
target_url

'https://www.1kkk.com/ch1-209332/'

In [82]:
res = htmlsession.get(target_url)

In [83]:
title

'寄宿学校的朱丽叶'

### 1. 爬取该chapter对应comic的标题, 进行筛选

In [84]:
tag = res.html.find('div.title', first=True)

In [85]:
chapter_title = tag.text.split()[1]
chapter_title

'寄宿学校的朱丽叶'

In [86]:
comic_title == chapter_title

True

### 2. 获取chapter长度

In [105]:
urls_page = []
for url in res.html.absolute_links:
    if url.startswith(target_url[: -1]):
        urls_page.append(url)

In [106]:
urls_page

['https://www.1kkk.com/ch1-209332-p50/',
 'https://www.1kkk.com/ch1-209332-p6/',
 'https://www.1kkk.com/ch1-209332-p5/',
 'https://www.1kkk.com/ch1-209332-p2/',
 'https://www.1kkk.com/ch1-209332-p4/',
 'https://www.1kkk.com/ch1-209332-p8/',
 'https://www.1kkk.com/ch1-209332-p7/',
 'https://www.1kkk.com/ch1-209332-p3/']

In [97]:
pattern_page_count = re.compile('p(\d+)\/')

In [107]:
chapter_page_count = max(map(int, [pattern_page_count.search(i).group(1) for i in urls_page]))
chapter_page_count

50

### 3. 尝试下载单页图片

In [111]:
index = 34
url_page = "".join([target_url[: -1], "-p", str(index)])
url_page

'https://www.1kkk.com/ch1-209332-p34'

In [133]:
res = htmlsession.get('https://www.1kkk.com/ch1-209332/#ipg34', headers=headers)

In [134]:
res.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html xmlns="http://www.w3.org/1999/xhtml"><head>    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />    <title>寄宿学校的朱丽叶漫画_第1话,_在线漫画阅读_极速漫画</title>    <meta name="Description" content="寄宿学校的朱丽叶1漫画简介：寄宿学校的朱丽叶漫画 ，2人的关系暴露就是死刑！秘密...极速漫画提供寄宿学校的朱丽叶1在线阅读和寄宿学校的朱丽叶68第一时间更新，同时也是国内寄宿学校的朱丽叶漫画最快最全的在线漫画阅读网站 - 68第1页。" />    <meta name="Keywords" content="寄宿学校的朱丽叶漫画,寄宿学校的朱丽叶第1话,寄宿学校的朱丽叶在线漫画" />    <meta name="Author" content="极速漫画:好漫画,为看漫画的人而生、1kkk.com" />    <!-- 样式引用 -->    <link rel="stylesheet" type="text/css" href="https://css99tel.cdndm5.com/v201808091405/blue/css/reset.css" />    <link rel="stylesheet" type="text/css" href="https://css99tel.cdndm5.com/v201808091405/blue/css/view.css" />    <link rel="shortcut icon" href="http://css99tel.cdndm5.com/v201808091405/blue/images/1kkk.ico" />    <script src="https://css99tel.cdndm5.com/v201808091405/blue/js/jqu

In [114]:
def save_image(img_url, order, save_path):
    img_response = htmlsession.get(img_url, headers=headers)
    with open(os.path.join(save_path, '%d.jpg' % order), 'ab') as f:
        f.write(img_response.content)

In [118]:
save_image(url_page, index, "E:/")